# Scrape Airline Routes and Times from OTAs

@Nileka add a description here as to what this notebook offers

### Execute the cell below once to turn off debugging

In [12]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

## Initialize the OTA scrape class

In [59]:
import os
import sys
from datetime import datetime, date, timedelta
#import configparser

# ROOT_DIR = "/home/nuwan/workspace/rezgate/wrangler"
# MODULE_DIR = os.path.join(ROOT_DIR, 'modules/ota/')
# DATA_DIR = os.path.join(ROOT_DIR, 'data/transport/airlines/scraper/')
# sys.path.insert(1,MODULE_DIR)
# import otaUtils as otau
sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
import rezaware as reza
from wrangler.modules.ota.scraper import airlineScrapers as air, scraperUtils as otasu
from utils.modules.etl.load import sparkwls as spark

if debug:
    import importlib
    reza = importlib.reload(reza)
    spark = importlib.reload(spark)
    otasu = importlib.reload(otasu)
    air = importlib.reload(air)
    
# kwargs = {
#     "ROOT_DIR":ROOT_DIR,   # absolute path to the wrangler dir
#     "MODULE_DIR":MODULE_DIR,   # absolute path to the ota module
#     "DATA_DIR":DATA_DIR, # absolute path to the scraper data dir
# }
''' optional - if not specified class will use the default values '''
prop_kwargs = {"WRITE_TO_FILE":True,   # necessary to emulate the etl dag
              }
clsScraper = air.AirlineScraper(name="kayak and momondo ota scrapes",**prop_kwargs)
clsSparkWL = spark.SparkWorkLoads(desc="ota airline segments", **prop_kwargs)

All python packages in rezaware loaded successfully!
All packages in load loaded successfully!
All scraper in ota software packages loaded successfully!
All scraper software packages loaded successfully!
Initialing scraper class for scraperUtils with instance Inheriting Utilities class for airline itinerary scraping
Initialing scraper class for airlineScrapers with instance data
Data path set to /home/nuwan/workspace/rezaware/wrangler/data/ota/scraper/transport/airlines/


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.8/logging/__init__.py", line 1085, in emit
    msg = self.format(record)
  File "/usr/lib/python3.8/logging/__init__.py", line 929, in format
    return fmt.format(record)
  File "/usr/lib/python3.8/logging/__init__.py", line 668, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.8/logging/__init__.py", line 373, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/nuwan/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/nuwan/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 976, in launch_instance
    app

## Initialize the set of OTA URL for Scraping

In [14]:
#path = DATA_PATH
file = "airOTAInputURLs.json"
start_date = date.today()
end_date = start_date + timedelta(days=1)
airportCodefile = "airportUSACities.csv"
# filePath=os.path.join(DATA_DIR, airportCodefile)

try:
    if start_date < date.today():
        raise ValueError("Start date must be greater than today: %s" % str(date.today()))
    if end_date <= start_date:
        raise ValueError("End date %s is invalid. It must be greater than Start Date: %s" % (str(end_date),str(start_date)))

    urls_kwargs = {
        "startDate": start_date,
        "endDate" : end_date,
#        'airportsFile' : airportCodefile,
        'codeColName' : 'airportCode',
        'departAirportCode': ['LAX','NYC','BOS'],
        'arriveAirportCode': ['LAS'],
    }

    _otaURLfilePath, _ota_url_parameterized_list  = clsScraper.build_scrape_url_info(
        file_name=file,
        dirPath=None,
        **urls_kwargs,
    )
    if _otaURLfilePath:
        print("Data in: %s" % _otaURLfilePath)
    if len(_ota_url_parameterized_list)>0:
        print("Completed parameterizing urls with %d instances." 
              % (len(_ota_url_parameterized_list)))

except Exception as err:
    _s_fn_id = "Class <WebScraper> Function <read_folder_csv_to_df>"
    print("[Error]"+_s_fn_id, err)
    print(traceback.format_exc())


Loaded 2 airline agencies to begin scraping OTA data.
Processing kayak.com ...
['LAX', 'NYC', 'BOS'] ['LAS']
Processing momondo.com ...
['LAX', 'NYC', 'BOS'] ['LAS']
Data in: /home/nuwan/workspace/rezaware/wrangler/data/ota/scraper/transport/airlines/tmp/scraper-build-scrape-url-list.csv
Completed parameterizing urls with 6 instances.


## Prepare folder & file structure

In [9]:
from datetime import datetime, timezone

dirPath = None
# dirPath=os.path.join(DATA_PATH, "itinerary/")
_search_dt = datetime.now()
_search_dt = _search_dt + (datetime.min - _search_dt) % timedelta(minutes=15)
''' include the timezone '''
_search_dt = (_search_dt.replace(tzinfo=timezone.utc)).isoformat()
# kwargs = {'searchDateTime': _search_dt,
#           'storageLocation': "local",   # values can be "local" or "AWS_S3"
#          }
search_kwargs = {
#    'SEARCH_DATETIME': _search_dt,
    'PARENT_DIR': "itinerary/",
    'STORAGE_METHOD': "local",   # values can be "local" or "AWS_S3"
}

_current_search_data_dir = clsScraper.make_storage_dir(**search_kwargs)
print("Extracting data into %s for search datetime: %s" % (_current_search_data_dir,str(_search_dt)))

Extracting data into /home/nuwan/workspace/rezaware/wrangler/data/ota/scraper/transport/airlines/itinerary/2022-10-25-11-30/ for search datetime: 2022-10-25T11:30:00+00:00


## Scrape data into CSVs
___TODO:___ pause if internet connection times out; might be possible with airflow.

In [16]:
import pandas as pd

if _otaURLfilePath:
    urlDF = pd.read_csv(_otaURLfilePath, sep=",")
    _otaURLParamDictList = urlDF.to_dict('records')

_l_saved_files = clsScraper.scrape_url_info(
    otaURLlist =_otaURLParamDictList,
#    otaURLfile = None,
    searchDT = _search_dt,
    data_store_dir =_current_search_data_dir)
print("Scraping completed and data saved in %s!" % _current_search_data_dir)

loading parameterized urls from list 6 records
[Error]function <_scrape_kayak_to_csv> No data received for https://www.momondo.com/flight-search/LAS-NYC/2022-10-25?a&fs=cfc=1;bfc=1;transportation=transportation_plane
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezaware/wrangler/modules/ota/scraper/airlineScrapers.py", line 555, in _scrape_kayak_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.momondo.com/flight-search/LAS-NYC/2022-10-25?a&fs=cfc=1;bfc=1;transportation=transportation_plane

[Error]function <_scrape_kayak_to_csv> No data received for https://www.momondo.com/flight-search/LAS-BOS/2022-10-25?a&fs=cfc=1;bfc=1;transportation=transportation_plane
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezaware/wrangler/modules/ota/scraper/airlineScrapers.py", line 555, in _scrape_kayak_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://

### Read CSV into spark dataframe

In [25]:
# dirPath = _current_search_data_dir
# dirPath = "/home/nuwan/workspace/rezgate/wrangler/data/transport/airlines/scraper/itinerary/2022-9-19-22-30/"
spark_kwargs = {"TO_PANDAS":True,   # change spark dataframe to pandas
                "IS_FOLDER":True,   # if folder then check if folder is empty
               }
_search_sdf, traceback = clsSparkWL.read_csv_to_sdf(
    filesPath=_current_search_data_dir,
    **spark_kwargs)
if not traceback:
    print("Spark loaded %d rows" % _search_sdf.shape[0])
else:
    print("No data loaded by spark; process failed!")
# _search_sdf = _search_sdf.distinct()

Spark loaded 65 rows


## Assign airpport city names

In [61]:
_aug_port_name_sdf = clsScraper.assign_lx_name(_search_sdf)
# columns_to_drop = ['flight_times','flight_type','booking_rate','airline']
# _aug_port_name_sdf=_aug_port_name_sdf.drop(*columns_to_drop)
_aug_port_name_sdf[:2]

,ota_name,search_dt,depart_port_code,arrive_port_code,flight_times,num_stops,duration,booking_rate,num_passengers,cabin_type,airline_name,depart_time,arrive_time,booking_price,currency,depart_port_name,depart_lx_type,arrive_port_name,arrive_lx_type
0,kayak.com,2022-10-25 19:30:00,BOS,LAS,9:43 am – 7:36 pm Delta,1 stop MSP,6h 53m LAS &dash; BOS,$503,1,$503,Delta,2022-10-25 09:43:00,2022-10-25 19:36:00,503,US$,Boston,city,NaN,city
1,kayak.com,2022-10-25 19:30:00,BOS,LAS,8:59 pm – 5:00 am +1 JetBlue,nonstop,5h 01m LAS &dash; BOS,$599,1,$599,JetBlue,2022-10-25 20:59:00,2022-10-25 05:00:00,599,US$,Boston,city,NaN,city


## Save cleaned SDF to Tmp File

In [62]:
_tmp_fname = clsSparkWL.save_sdf_to_csv(_aug_port_name_sdf)
print("%d rows saved to %s" % (_aug_port_name_sdf.shape[0], _tmp_fname))

97 rows saved to /home/nuwan/workspace/rezaware/utils/data/etl/load/tmp/load_save_sdf_to.csv


## Read CSV from Tmp and Save to DB Table

In [64]:
''' Save dataframe to table '''
_s_tbl_name = "ota_airline_routes"
_get_tmp_sdf, traceback = clsSparkWL.read_csv_to_sdf(filesPath=_tmp_fname)
_get_tmp_sdf.printSchema()
count = clsSparkWL.insert_sdf_into_table(save_sdf=_get_tmp_sdf, dbTable=_s_tbl_name)
print("%d Data saved to %s" % (count,_s_tbl_name))

root
 |-- ota_name: string (nullable = true)
 |-- search_dt: timestamp (nullable = true)
 |-- depart_port_code: string (nullable = true)
 |-- arrive_port_code: string (nullable = true)
 |-- flight_times: string (nullable = true)
 |-- num_stops: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- booking_rate: string (nullable = true)
 |-- num_passengers: integer (nullable = true)
 |-- cabin_type: string (nullable = true)
 |-- airline_name: string (nullable = true)
 |-- depart_time: timestamp (nullable = true)
 |-- arrive_time: timestamp (nullable = true)
 |-- booking_price: integer (nullable = true)
 |-- currency: string (nullable = true)
 |-- depart_port_name: string (nullable = true)
 |-- depart_lx_type: string (nullable = true)
 |-- arrive_port_name: double (nullable = true)
 |-- arrive_lx_type: string (nullable = true)

Wait a moment while we insert data int ota_airline_routes


Save to ota_airline_routes complete!
97 Data saved to ota_airline_routes


## Read data from table

In [65]:
_s_tbl_name = "ota_airline_routes"
data = clsSparkWL.get_data_from_table(dbTable=_s_tbl_name)
print("%d records loaded from %s" %(data.count(),_s_tbl_name))
data.select("*").distinct().sort("search_dt").show(3, vertical=True, truncate=False)

Wait a moment, retrieving data ...
Loading complete!
97 records loaded from ota_airline_routes
22/10/25 15:14:51 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
-RECORD 0-----------------------------------------------
 uuid             | null                                
 ota_name         | kayak.com                           
 search_dt        | 2022-10-25 19:30:00                 
 agent_name       | null                                
 depart_port_name | Boston                              
 depart_port_code | BOS                                 
 depart_lx_type   | city                                
 arrive_port_name | NaN                                 
 arrive_port_code | LAS                                 
 arrive_lx_type   | city                                
 flight_times     | 6:00 am – 5:11 pm United Airlines   
 depart_time      | 2022-10-25 06:00:00  

In [115]:
_tmp_sdf = data.select("*").distinct().sort("search_dt")
_tmp_fname = clsSparkWL.save_sdf_to_csv(_tmp_sdf, filesPath='/tmp/ota_prices.csv')
print(_tmp_fname)

/tmp/ota_prices.csv


In [278]:
'''DEPRECATED moved to function in airlineScraper'''
import os
''' Get airport codes dictionary '''
airportCodefile = "airportUSACities.csv"
airportsPath = os.path.join(DATA_PATH,'airports/')
airports_sdf = clsSparkWL.read_csv_to_sdf(filesPath=airportsPath)
airports_sdf = airports_sdf.withColumn("airportCode", trim(airports_sdf.airportCode))

_search_sdf = _search_sdf.withColumn("arrive_port_code", trim(_search_sdf.arrive_port_code)) \
                        .withColumn("depart_port_code", trim(_search_sdf.depart_port_code))\

''' first do the departure codes and names '''
dep_airports_sdf = airports_sdf.selectExpr("city as depart_port_name", \
                                       "airportCode as depart_port_code", \
                                      )
''' Lookup & augment departure name '''
aug_depart_sdf = dep_airports_sdf.join(_search_sdf,"depart_port_code")

''' next do the same for arrival codes and names '''
arr_airports_sdf = airports_sdf.selectExpr("city as arrive_port_name", \
                                       "airportCode as arrive_port_code", \
                                      )
''' Lookup & augment arrival name '''
aug_arr_sdf = arr_airports_sdf.join(aug_depart_sdf,on="arrive_port_code")

# aug_arr_sdf = _search_sdf.join(arr_airports_sdf,
#                                     trim(arr_airports_sdf['arrive_port_code']) == trim(_search_sdf['arrive_port_code'])
#                        ,'inner')
#                                    ).select(_search_sdf['arrive_port_name'].alias('arrive_port_name')
#                    ,arr_airports_sdf['users_count'])

arr_airports_sdf.printSchema()
#destinations_sdf.show(n=2, vertical=True, truncate=False)
print("airports dictionary loaded!")

#aug_search_sdf = airports_sdf.join(_search_sdf,"arrive_port_code")
#aug_search_sdf.show(n=2, vertical=True, truncate=False)
print("Destination names augmented to dataframe!")

root
 |-- arrive_port_name: string (nullable = true)
 |-- arrive_port_code: string (nullable = true)

airports dictionary loarded!
Destination names augmented to dataframe!
